In [ ]:
#@markdown # 1. Setup
!git clone --branch dev --quiet https://github.com/pytti-tools/blended-diffusion
!git clone --branch dev --quiet https://github.com/pytti-tools/pytti-notebook
!pip install --quiet ftfy regex lpips kornia ipycanvas

!mkdir checkpoints # put 'http://batbot.ai/models/blended-diffusion/256x256_diffusion_uncond.pt' into checkpoints folder
!wget 'http://batbot.ai/models/blended-diffusion/256x256_diffusion_uncond.pt' 
!mv 256x256_diffusion_uncond.pt checkpoints/

from google.colab import output
output.enable_custom_widget_manager()

# I am not proud.
%cd ./pytti-notebook/src 
from pytti_notebook.widgets import Sketcher
%cd ../..

In [ ]:
#@markdown # 2.1 Settings

#@markdown ## Basic Settings

#@markdown `image_path` Specifies the initial image. Can be image URL or path to locally uploaded image  

#@markdown `mask_path` (Optional) Specifies the mask image. Must be a local path if provided. If not provided, a "sketcher" UI will be loaded in which you can freehand a mask  

#@markdown `out_path` Specifies the parent output folder. Outputs will be saved to a subdirectory named "ranked"  

#@markdown `text_prompt`: Provide text to be used as a semantic target for the diffusion process over the masked area of the initial image  

#@markdown `batch_size`: Number of images to sample (i.e. generate) per diffusion process. Total number of generated outputs will be `batch_size * aug_num`

#@markdown `aug_num`: Number of augmentations to perform. Total number of generated outputs will be `batch_size * aug_num`

#@markdown `invert_mask`: Check this box if your mask image uses white to denote the fixed region and black to denote the modifiable region.

text_prompt = "a ball of yarn" # @param{'type':'string'}
text_prompt = f"'{text_prompt}'" # wrap prompt in quotes for CLI

image_path = "/content/blended-diffusion/input_example/img.png" # @param{'type':'string'}
mask_path = "" # @param{'type':'string'}
out_path = "/content" # @param{'type':'string'}
batch_size = 2 # @param{'type':'integer'}
aug_num = 2  # @param{'type':'integer'}
invert_mask = False # @param{'type':'boolean'}

#@markdown ## Advanced Settings

skip_timesteps = 25




In [ ]:
#@markdown # 2.2 Sketch a Mask

#@markdown Use your mouse to draw a mask onto the image. Recommend that you draw slowly. 
#@markdown Also, it helps if you draw an "unnecessary" region inside your mask as your last action, as the last drawing action sometimes doesn't register correctly.
#@markdown When you are done, use the "save" button to write the mask to disk and check that it looks right using the cell below.

import warnings

if len(mask_path) > 0:
    warnings.warn(
        "It looks like you already specified a mask path above. "
        "If you want to use that as the mask, you shouldn't run this cell. "
        "Rerun the cell above (step 2.1) to ensure your mask was specified correctly."
    )

mask_path = "_mask.png"
invert_mask=True
widget = Sketcher(
    background_image=image_path,
    starting_color="#000000",
    )
widget.show()
widget._set_background()

In [ ]:
#@markdown # 2.3 Check your Mask
#@markdown The sketcher is funny and sometimes doesn't register the last action you took.
#@markdown You can double check your mask saved properly here. 
#@markdown If it didn't, you can probably fix it by just drawing anything inside your mask region and saving again.

from IPython.display import Image
Image(mask_path)

In [ ]:

# blended-diffusion/optimization/arguments.py

if not invert_mask:

    !python '/content/blended-diffusion/main.py' \
        --prompt {text_prompt} \
        --init_image {image_path}  \
        --mask {mask_path} \
        --output_path {out_path} \
        --batch_size {batch_size} \
        --aug_num {aug_num}

else:

    !python '/content/blended-diffusion/main.py' \
        --prompt {text_prompt} \
        --init_image {image_path}  \
        --mask {mask_path} \
        --output_path {out_path} \
        --batch_size {batch_size} \
        --aug_num {aug_num} \
        --invert_mask
